# Import Library

In [1]:
import logging
import math
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional
import json
from itertools import chain

import datasets
import evaluate
import torch
from datasets import load_dataset

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    AdapterTrainer,
    TrainingArguments,
    default_data_collator,
    is_torch_tpu_available,
    set_seed,
    AdapterArguments,
    AdapterTrainer
)
from transformers.testing_utils import CaptureLogger
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers import EarlyStoppingCallback
from transformers.adapters import (
    GPT2AdapterModel, AutoAdapterModel,  
    AdapterConfig, PrefixTuningConfig, HoulsbyConfig, LoRAConfig
)

/rds/general/user/jj1122/home/anaconda3/envs/m2d2_adapter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from os import listdir

# Config

In [25]:
# job_cd = sys.argv[2] # TODO
# print("job_cd: ", job_cd)
prj_path = "/rds/general/user/jj1122/home/projects/domain_adaptation/"
cache_dir="/rds/general/user/jj1122/ephemeral/.cache/huggingface"
model_path_format = "/rds/general/user/jj1122/home/projects/m2d2/dataset/{job_cd}/"

job_cd = "bottleneck_adapter___Religion_and_belief_systems"

In [26]:
device_id = 'cpu'
n_layers = 12
list_modules = ['ln_1', 'attn', 'ln_2', 'mlp']
trace_module_id = "transformer.h.{l}.{m}"

In [27]:
list_models = [
    "cs_l1",
    "nlin_l1",
    "Health_and_fitness",
    "Natural_and_physical_sciences",
    "Religion_and_belief_systems",
    "Culture_and_the_arts",
    "General_referece",
    "econ_l1",
    "History_and_events",
    "Human_activites",
    "Mathematics_and_logic",
    "astro-ph_l1",
    "cond-mat_l1",
#     "eess_l1 (No Metric)",
    "math_l1",
#     "physics_l1 (ERROR)",
    "q-bio_l1",
#     "q-fin_l1 (No Metric)",
    "stat_l1",
    "Philosophy",
    "Philosophy_and_thinking",
    "Society_and_social_sciences",
    "Technology_and_applied_sciences",
#     "art_l1 (No Data)"
]
len(list_models), len(set(list_models))

(20, 20)

In [21]:
# def get_model_path(model_id):
#     model_path_format = "/rds/general/user/jj1122/home/projects/m2d2/dataset/{model_id}/models"
#     ckpt_path_format = "/checkpoint-{ckpt}"

#     if model_id == "gpt2":
#         model_path = "gpt2"
# #         ckpt = "zs"
#     else:
#         model_path = model_path_format.format(model_id=model_id)
#         l_dir = listdir(model_path)

#         if all([len(x.split(".")) == 1 for x in l_dir]):
#             ckpt = max([int(x.split("-")[1]) for x in l_dir])
#             model_path += ckpt_path_format.format(ckpt=ckpt)
# #         else:
# #             ckpt = "final"
#     return model_path

# config_models = {model_id: {} for model_id in list_models}
# for model_id in list_models:
#     config_models[model_id]["model_path"] = get_model_path(model_id)
#     json_path = config_models[model_id]["model_path"] + "/trainer_state.json"
#     with open(json_path, "r") as json_file:
#         trainer_state = json.load(json_file)
#         config_models[model_id]["val_ppl"] = math.exp(trainer_state["best_metric"])
#         config_models[model_id]["train_steps"] = int(trainer_state["best_model_checkpoint"].split("/")[-1].split("-")[-1])
#         config_models[model_id]["train_eps"] = float(trainer_state["epoch"])

In [22]:
from transformers import GPT2LMHeadModel

In [28]:
model_path_format.format(job_cd=)

'/rds/general/user/jj1122/home/projects/m2d2/dataset/bottleneck_adapter___Religion_and_belief_systems/'

In [42]:
base_model =  GPT2LMHeadModel.from_pretrained("gpt2", cache_dir=cache_dir)
tuned_model_1 = GPT2AdapterModel.from_pretrained("gpt2", cache_dir=cache_dir)
tuned_model_1.add_adapter(model_path_format.format(job_cd=job_cd))
tuned_model_2 = GPT2AdapterModel.from_pretrained("gpt2", cache_dir=cache_dir)
tuned_model_2.add_adapter(model_path_format.format(job_cd="bottleneck_adapter___History_and_events"))

Some weights of GPT2AdapterModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.11.attn.masked_bias', 'h.2.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.4.attn.masked_bias', 'h.10.attn.masked_bias', 'h.6.attn.masked_bias', 'h.0.attn.masked_bias', 'h.3.attn.masked_bias', 'h.7.attn.masked_bias', 'h.5.attn.masked_bias', 'h.1.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2AdapterModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.11.attn.masked_bias', 'h.2.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.4.attn.masked_bias', 'h.10.attn.masked_bias', 'h.6.attn.masked_bias', 'h.0.attn.masked_bias', 'h.3.attn.masked_bias', 'h.7.attn.masked_bias', 'h.5.attn.masked_bias', 'h.1.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [43]:
tuned_model_1

GPT2AdapterModel(
  (shared_parameters): ModuleDict()
  (transformer): GPT2Model(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): MergedLinear(
            in_features=768, out_features=2304, bias=True
            (loras): ModuleDict()
          )
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (prefix_tuning): PrefixTuningShim(
            (prefix_gates): ModuleDict()
            (pool): PrefixTuningPool(
              (prefix_tunings): ModuleDict()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
  

In [53]:
for tup1, tup2 in zip(tuned_model_1.named_parameters(), tuned_model_2.named_parameters()):
    if "adapters" in tup1[0] and "bias" in tup1[0]:
        print(tup1[0], torch.all(tup1[1] == tup2[1]))
        print(tup1[1], tup2[1])
        break
        

transformer.h.0.output_adapters.adapters./rds/general/user/jj1122/home/projects/m2d2/dataset/bottleneck_adapter___Religion_and_belief_systems/.adapter_down.0.bias tensor(True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True) Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)


In [ ]:
tuned_model = AutoAdapterModel(model_path_format.format(job_cd=job_cd), cache_dir=cache_dir).to(device_id)